In [7]:
import depthai as dai
import cv2
import numpy as np

# Pipeline configuration
pipeline = dai.Pipeline()

# Configure stereo depth stream
stereo = pipeline.createStereoDepth()
stereo.setConfidenceThreshold(200)
stereo.setLeftRightCheck(False)
stereo.setExtendedDisparity(False)
stereo.setSubpixel(False)

# Configure RGB camera stream
camRgb = pipeline.createColorCamera()
camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
camRgb.setResolution(1920, 1080)  # Set resolution to 1920x1080
camRgb.setInterleaved(False)
camRgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Link streams
stereo.depth.link(camRgb.inputControl)

# Create output queues
depthQueue = pipeline.createXLinkOut()
depthQueue.setStreamName("depth")

rgbQueue = pipeline.createXLinkOut()
rgbQueue.setStreamName("rgb")

# Connect to the device
with dai.Device(pipeline) as device:
    # Output queues
    depthQueue = device.getOutputQueue(name="depth", maxSize=1, blocking=False)
    rgbQueue = device.getOutputQueue(name="rgb", maxSize=1, blocking=False)

    # Initialize object tracker
    tracker = cv2.TrackerCSRT_create()

    # Main loop
    while True:
        # Get frames from the Oak-D camera
        depthFrame = depthQueue.get().getFrame()
        rgbFrame = rgbQueue.get().getCvFrame()

        # Object detection and tracking (placeholder)
        # Here, you can implement your object detection and tracking algorithm
        # For demonstration, we'll use a dummy bounding box
        bbox = (100, 100, 200, 200)  # Format: (x, y, width, height)

        # Initialize tracker with the bounding box in the first frame
        if bbox is not None:
            tracker.init(rgbFrame, bbox)

        # Update tracker in subsequent frames
        success, bbox = tracker.update(rgbFrame)

        # Draw bounding box around the tracked object
        if success:
            x, y, w, h = [int(coord) for coord in bbox]
            cv2.rectangle(rgbFrame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Stereo vision and depth estimation (placeholder)
        # Here, you can implement stereo vision techniques to estimate depth
        # For demonstration, we'll use a dummy depth map
        depth_map = np.zeros_like(rgbFrame)  # Placeholder for depth map

        # Dimension estimation (placeholder)
        # Once an object is detected and tracked, estimate its dimensions using the depth information

        # Display the frames with detected objects and dimensions
        cv2.imshow('RGB Camera', rgbFrame)
        cv2.imshow('Depth Map', depth_map)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the camera and close all windows
cv2.destroyAllWindows()


C:\Users\azuma\AppData\Local\Temp\ipykernel_41472\415712182.py:10: DeprecationWarning: setConfidenceThreshold() is deprecated, Use 'initialConfig.setConfidenceThreshold()' instead
  stereo.setConfidenceThreshold(200)
C:\Users\azuma\AppData\Local\Temp\ipykernel_41472\415712182.py:17: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)


TypeError: setResolution(): incompatible function arguments. The following argument types are supported:
    1. (self: depthai.node.ColorCamera, resolution: depthai.ColorCameraProperties.SensorResolution) -> None

Invoked with: <depthai.node.ColorCamera object at 0x0000020FCBE78270>, 1920, 1080

In [1]:
## Question 2
import cv2
import open3d as o3d
import depthai as dai
from slam import process
from display import Display
from pointmap import PointMap


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:

# Create a pipeline
pipeline = dai.Pipeline()

# Define a source - color camera
camRgb = pipeline.createColorCamera()
camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_4_K)
camRgb.setVideoSize(1000, 1000)
camRgb.setInterleaved(False)
camRgb.setColorOrder(dai.ColorCameraProperties.ColorOrder.RGB)

# Create outputs
xoutRgb = pipeline.createXLinkOut()
xoutRgb.setStreamName("rgb")
camRgb.preview.link(xoutRgb.input)

xoutRgbVideo = pipeline.createXLinkOut()
xoutRgbVideo.setStreamName("video")
xoutRgbVideo.input.setBlocking(False)
xoutRgbVideo.input.setQueueSize(1)
camRgb.video.link(xoutRgbVideo.input)

# Create objects for point mapping and display
pmap = PointMap()
display = Display()
pcd = o3d.geometry.PointCloud()
visualizer = o3d.visualization.Visualizer()
visualizer.create_window(window_name="3D plot", width=960, height=540)

# Connect to the device
with dai.Device() as device:
    print('Connected cameras: ', device.getConnectedCameras())
    device.startPipeline(pipeline)

    # Output queues
    qRgb = device.getOutputQueue(name="rgb", maxSize=30, blocking=False)
    out = device.getOutputQueue(name="video", maxSize=1, blocking=False)

    while True:
        frame = qRgb.get()
        output = out.get()
        frame = output.getCvFrame()

        img, tripoints, kpts, matches = process(frame)
        xyz = pmap.collect_points(tripoints)

        if kpts is not None or matches is not None:
            display.display_points2d(frame, kpts, matches)

        display.display_vid(frame)

        if xyz is not None:
            display.display_points3d(xyz, pcd, visualizer)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

cv2.destroyAllWindows()


C:\Users\azuma\AppData\Local\Temp\ipykernel_35508\2085936821.py:13: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)


Connected cameras:  [<CameraBoardSocket.CAM_A: 0>, <CameraBoardSocket.CAM_B: 1>, <CameraBoardSocket.CAM_C: 2>]


: 